## Importing Libraries, Creating Dataframe & Current Time

In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import matplotlib.pyplot as plt

In [2]:
df=pd.DataFrame(columns=['Symbol','Type','Indicative Entry Price','Entry Price','Exit Price','PnL (%)'])

## Inputing Stocks With Good & Bad News

In [3]:
good_news,bad_news=[],[]
a=int(input('Enter number of stocks with good news'))
b=int(input('Enter number of stocks with bad news'))
for i in range(a):
    c=input(f'Enter stock symbol {i+1} with good news')
    good_news.append(c)
for i in range(b):
    d=input(f'Enter stock symbol {i+1} with bad news')
    bad_news.append(d)
common=good_news+bad_news

## Fetching Latest Price, % Change, Total Buy & Sell Quantity, Open Price

In [4]:
def orderbook(stock):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': f'https://www.nseindia.com/get-quotes/equity?symbol={stock}',
    })
    try:
        main_page = session.get(f'https://www.nseindia.com/get-quotes/equity?symbol={stock}', timeout=10)
        api_url = f'https://www.nseindia.com/api/quote-equity?symbol={stock}'
        response = session.get(api_url, timeout=10)
        response.raise_for_status()
        lastPrice = response.json()['priceInfo']['lastPrice']
        pChange = round(response.json()['priceInfo']['pChange'],2)
        totalBuy = response.json()['preOpenMarket']['totalBuyQuantity']
        totalSell = response.json()['preOpenMarket']['totalSellQuantity']
        return [lastPrice, pChange, totalBuy, totalSell]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

## Placing Order After 9:06:40 AM

In [ ]:
now=datetime.datetime.now()
hour,minute,second=now.hour,now.minute,now.second
if (hour==9 and minute==6 and second>40):
    for i in range(len(common)):
        lastPrice,pChange,totalBuy,totalSell= orderbook(common[i]) 
        if common[i] in good_news:
            if (totalBuy>1.5*totalSell and pChange>1 and pChange<4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Buy'], 'Indicative Entry Price': [int(1.08*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
            elif (totalBuy>1.5*totalSell and pChange>4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Sell'], 'Indicative Entry Price':[int(0.92*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
            elif (totalSell>1.5*totalBuy and pChange>4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Sell'], 'Indicative Entry Price':[int(0.92*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
            elif (totalSell>1.5*totalBuy and pChange>1 and pChange<4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Sell'], 'Indicative Entry Price':[int(0.92*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
        else:
            if (totalSell>1.5*totalBuy and pChange<-1 and pChange>-4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Sell'], 'Indicative Entry Price':[int(0.92*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
            elif (totalBuy>1.5*totalSell and pChange<-1 and pChange>-4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Buy'], 'Indicative Entry Price': [int(1.08*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
            elif (totalBuy>1.5*totalBuy and pChange<-4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Buy'], 'Indicative Entry Price': [int(1.08*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
            elif (totalSell>1.5*totalBuy and pChange<-4):
                new_row=pd.DataFrame({'Symbol': [f'{common[i]}'], 'Type': ['Buy'], 'Indicative Entry Price': [int(1.08*lastPrice)]})
                df=pd.concat([df, new_row],ignore_index=True)
else:
    print('Not the correct time')

## Fetching Open (9:08 AM) & Exit Price

In [5]:
def open_or_exit(stock):
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': f'https://www.nseindia.com/get-quotes/equity?symbol={stock}',
    })
    try:
        main_page = session.get(f'https://www.nseindia.com/get-quotes/equity?symbol={stock}', timeout=10)
        api_url = f'https://www.nseindia.com/api/quote-equity?symbol={stock}'
        response = session.get(api_url, timeout=10)
        response.raise_for_status()
        exit = response.json()['priceInfo']['lastPrice']
        return [exit]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [11]:
now=datetime.datetime.now()
hour,minute,second=now.hour,now.minute,now.second
if (hour==9 and minute==8):
    for i in range(len(common)):
        result_open=open_or_exit(common[i])
        df.loc[df['Symbol']==common[i],'Entry Price']=result_open
else:
    print('Not the correct time')

In [ ]:
for i in range(len(common)):
    result_close=open_or_exit(common[i])
    df.loc[df['Symbol']==common[i],'Exit Price']=result_close

## Calculating PnL (%) & Saving Data

In [17]:
df['Entry Price']=df['Entry Price'].astype(float)
df['Exit Price']=df['Exit Price'].astype(float)

In [18]:
if not df.empty:
    for i in range(len(common)):
        df.loc[(df['Symbol']==common[i]) & (df['Type']=='Buy'),'PnL (%)']=round(((df['Exit Price']-df['Entry Price'])*100/df['Entry Price']),2)
        df.loc[(df['Symbol']==common[i]) & (df['Type']=='Sell'),'PnL (%)']=round(((df['Entry Price']-df['Exit Price'])*100/df['Entry Price']),2)

In [20]:
if not df.empty:
    df.to_csv("Pre Market.csv",mode='a',header=False,index=False)